In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from prefect import Flow, Parameter, unmapped
import pandas as pd
from prefect.engine.executors import DaskExecutor
from crawto.ml_flow import data_cleaning_flow
from tinydb import TinyDB

In [4]:
input_df = pd.read_csv("data/house-prices-advanced-regression-techniques/train.csv")
test= pd.read_csv("data/house-prices-advanced-regression-techniques/test.csv")

In [5]:
executor = DaskExecutor()
data_cleaner = data_cleaning_flow.run(
    input_data=input_df,
    problem="regression",
    target="SalePrice",
    features="infer",
    executor=executor,
    db = "crawto.db"
)

[2020-04-24 23:42:28,845] INFO - prefect.FlowRunner | Beginning Flow run for 'data_cleaning'
[2020-04-24 23:42:28,850] INFO - prefect.FlowRunner | Starting flow run.
[2020-04-24 23:42:29,100] INFO - prefect.TaskRunner | Task 'features': Starting task run...
[2020-04-24 23:42:29,115] INFO - prefect.TaskRunner | Task 'features': finished task run for task with final state: 'Success'
[2020-04-24 23:42:29,136] INFO - prefect.TaskRunner | Task 'db': Starting task run...
[2020-04-24 23:42:29,169] INFO - prefect.TaskRunner | Task 'target': Starting task run...
[2020-04-24 23:42:29,190] INFO - prefect.TaskRunner | Task 'db': finished task run for task with final state: 'Success'
[2020-04-24 23:42:29,224] INFO - prefect.TaskRunner | Task 'target': finished task run for task with final state: 'Success'
[2020-04-24 23:42:29,415] INFO - prefect.TaskRunner | Task 'input_data': Starting task run...
[2020-04-24 23:42:29,417] INFO - prefect.TaskRunner | Task 'problem': Starting task run...
[2020-04-24

e:\projects\crawto\.venv\lib\site-packages\sklearn\preprocessing\_data.py:2982: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


[2020-04-24 23:42:31,172] INFO - prefect.TaskRunner | Task 'fit_target_encoder': finished task run for task with final state: 'Success'
[2020-04-24 23:42:31,193] INFO - prefect.TaskRunner | Task 'target_encoder_transform': Starting task run...
[2020-04-24 23:42:31,206] INFO - prefect.TaskRunner | Task 'target_encoder_transform': Starting task run...
[2020-04-24 23:42:31,396] INFO - prefect.TaskRunner | Task 'target_encoder_transform': finished task run for task with final state: 'Success'
[2020-04-24 23:42:31,424] INFO - prefect.TaskRunner | Task 'target_encoder_transform': finished task run for task with final state: 'Success'
[2020-04-24 23:42:31,439] INFO - prefect.TaskRunner | Task 'fit_yeo_johnson_transformer': finished task run for task with final state: 'Success'
[2020-04-24 23:42:31,464] INFO - prefect.TaskRunner | Task 'transform_yeo_johnson_transformer': Starting task run...
[2020-04-24 23:42:31,472] INFO - prefect.TaskRunner | Task 'transform_yeo_johnson_transformer': Starti

In [ ]:
from prefect.engine.executors import DaskExecutor
from crawto.meta_model import MetaModel, meta_model_flow
meta = MetaModel("regression", TinyDB("db.json"), use_default_models=True)
models = meta.models
executor = DaskExecutor()
meta_model_run = meta_model_flow.run(
    train_data = "transformed_train.df",
    valid_data = "transformed_valid.df",
    train_target = "train_target.df",
    db_name="crawto.db",
    executor=executor,
)

In [ ]:
with Flow("data_visualization") as flow:
    transformed_train_df = pd.read_feather("transformed_train.df")
    transformed_valid_df = pd.read_feather("transformed_valid.df")
    svd = fit_svd(transformed_train_df)
    svd_train = svd_transform(svd, transformed_train_df, "transformed_train_df",tinydb)
    svd_valid = svd_transform(svd, transformed_valid_df, "transformed_valid_df",tinydb)
    